In [1]:
using Meiran
using LinearAlgebra
using Tonari
using Plots
using Random

rng = MersenneTwister(1234);

Define the power spectral densities for the two processes and the cross-spectral density between them 

In [2]:
p1 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
p2 = SingleBendingPowerLaw(1.0, 0.550, 8e-2, 2.95)
Δϕ = ConstantTimeLag(5.4)
cs = CrossSpectralDensity(p1, p2, Δϕ)

T = 150.0
Δt = 1.0
t = collect(0.0:Δt:T-Δt)
S_low, S_high = 5, 5

f0, fM = 1 / T / S_low, 1 / (2Δt) * S_high
J = 100

100

In [4]:
σ_X₁², σ_X₂² = zeros(length(t)), zeros(length(t))
Σ = BlockMatrix_from_cs(cs, t, t, σ_X₁², σ_X₂², f0, fM, J)


BlockCovarianceMatrix([0.12057264188482204 0.11699276845270255 … 0.0008531520424300245 0.0008792484670302613; 0.11699276845270255 0.12057264188482204 … 0.0008116777501405635 0.0008531520424300245; … ; 0.0008531520424300245 0.0008116777501405635 … 0.12057264188482204 0.11699276845270255; 0.0008792484670302613 0.0008531520424300245 … 0.11699276845270255 0.12057264188482204], [0.10344323746852452 0.12033635533106725 … 0.002084857648477922 0.002001684517981912; 0.08896412289743097 0.10344323746852452 … 0.00222131510864581 0.002084857648477922; … ; 0.0016990850540399738 0.001849789994110569 … 0.10344323746852452 0.12033635533106725; 0.001596064398706203 0.0016990850540399738 … 0.08896412289743097 0.10344323746852452], [0.34572893124857307 0.31131955717655263 … 0.005088335126957007 0.005254121552299166; 0.31131955717655263 0.34572893124857307 … 0.004909875439662378 0.005088335126957007; … ; 0.005088335126957007 0.004909875439662378 … 0.34572893124857307 0.31131955717655263; 0.005254121552299

In [5]:
sanity_checks(Σ)

In [6]:
L₁₁, L₂₁, L₂₂,S = cholesky(Σ)

([0.34723571516308926 0.0 … 0.0 0.0; 0.33692608030759025 0.083984869431257 … 0.0 0.0; … ; 0.0024569824046737736 -0.00019222153895354237 … 0.06318198231790434 0.0; 0.0025321371870324367 1.0698596257797622e-7 … 0.10521324484252159 0.06318161148347952], [0.2979049474214927 0.2377143553035603 … 0.00034720313392875825 0.00033353474043863084; 0.2562067178361719 0.2038523414449525 … 0.0005325121391976749 0.000385614139099571; … ; 0.0048931748084782975 0.002395095533187421 … 0.1688402381586053 0.19429974298500088; 0.0045964868503130946 0.0017909030209508584 … 0.1444298478140871 0.16884148660276127], [0.07972462904051697 0.0 … 0.0 0.0; -0.0009368471713101003 0.0789274864626453 … 0.0 0.0; … ; 0.0016629691531214695 0.002644906461531324 … 0.20142239553672267 0.0; 0.00159482249672254 0.0024269263455462798 … 0.17206464191466164 0.2157426250863336], [0.006356016475648041 -7.46897932003554e-5 … 0.00013257959883843182 0.0001271466319366756; -7.46897932003554e-5 0.006230425801933448 … 0.0002071978709903

In [ ]:
n_samples = 50
y = randn(rng, length(t), 2,n_samples)
y₁, y₂ = y[:,1,:], y[:,2,:]

In [41]:
x₁_sim = [L₁₁ * y₁[:,i] for i in 1:n_samples];
x₂_sim = [L₂₁ * y₁[:,i] + L₂₂ * y₂[:,i]  for i in 1:n_samples];

In [ ]:
k = 1
scatter(t,x₁_sim[k],label="1", color="blue",framestyle=:box,size=(900,600),xlabel="Time",ylabel="Value")
plot!(t,x₁_sim[k],label="1", color="blue", linestyle=:dash)
scatter!(t,x₂_sim[k],label="2", color="red")
plot!(t,x₂_sim[k],label="2", color="red", linestyle=:dash)

In [ ]:
f, γ², Δφ, γ²_err, Δφ_err, Δτ, Δτ_err, P̄₁, P̄₂, C̄ = cross_periodogram(t, mapreduce(permutedims,vcat,x₁_sim)', mapreduce(permutedims,vcat,x₂_sim)')

In [ ]:
plot(f,P̄₁/length(f)/2,label="Periodogram P₁",xscale=:log10,yscale=:log10)
plot!(f,P̄₂/length(f)/2,xscale=:log10,yscale=:log10,label="Periodogram P₂",xlabel="Frequency (d^-1)",ylabel="Power")
plot!(f,p1(f),xscale=:log10,yscale=:log10,label="True Power spectrum 1")
plot!(f,p2(f),xscale=:log10,yscale=:log10,label="True Power spectrum 2")

In [ ]:
l = @layout [a; b]
pl1 = scatter(f, γ², yerr=γ²_err, label="γ²", xscale=:log10,ylims=(0,1.1),ms=4,ylabel="Coherence",link=:x, framestyle=:box)
pl1 = hline!([1.0],color=:black,label="True coherence")
pl2 = scatter(f, Δτ, yerr=Δτ_err, xscale=:log10, ylabel="Time delay (d)",xlabel="Frequency (d^-1)",link=:x, framestyle=:box,label=nothing)
pl2 = hline!([-cs.Δφ.Δτ ],color=:black,label="True delay")
pl2 = vline!([1/cs.Δφ.Δτ ],color=:black,label="Limit frequency")
plot(pl1, pl2, layout=l,size=(800,600))